In [1]:
# Import Dependencies
import pandas as pd
from pathlib import Path

# BEA Personal Consumption

In [2]:
bea_personal_consumption_csv = Path("./source/BEA/Personal_consumption_by_state_2020_2022.csv")

# Skip the first 3 lines
source_df = pd.read_csv(bea_personal_consumption_csv, header=3)

# Remove rows with NaN, since they are related to desciptions
source_df.dropna(inplace=True)

source_df.head(25)

,GeoFips,GeoName,LineCode,Description,2020,2021,2022
0,01000,Alabama,1.0,Per capita personal consumption expenditures,34878.0,39174.0,42391.0
1,01000,Alabama,2.0,Goods,13131.0,15275.0,16468.0
2,01000,Alabama,3.0,Durable goods,4247.0,5186.0,5500.0
3,01000,Alabama,4.0,Motor vehicles and parts,1769.0,2277.0,2352.0
4,01000,Alabama,5.0,Furnishings and durable household equipment,989.0,1126.0,1201.0
5,01000,Alabama,6.0,Recreational goods and vehicles,1009.0,1190.0,1336.0
6,01000,Alabama,7.0,Other durable goods,480.0,593.0,611.0
7,01000,Alabama,8.0,Nondurable goods,8883.0,10088.0,10968.0
8,01000,Alabama,9.0,Food and beverages purchased for off-pre...,3196.0,3495.0,3708.0
9,01000,Alabama,10.0,Clothing and footwear,976.0,1221.0,1285.0


In [3]:
# Keep only required LineCodes, related to main groups:
# 1 - Total
# 3 - Durable goods
# 8 - Nondurable goods
# 14 - Services
# 22 - NPISHs

source_df = source_df[(source_df["LineCode"].isin([1, 3, 8, 14, 22]))]

# Remove unnecessary columns
source_df.drop(columns=["GeoFips", "Description"], inplace=True)

source_df.head(10)

,GeoName,LineCode,2020,2021,2022
0,Alabama,1.0,34878.0,39174.0,42391.0
2,Alabama,3.0,4247.0,5186.0,5500.0
7,Alabama,8.0,8883.0,10088.0,10968.0
13,Alabama,14.0,21099.0,23412.0,25297.0
21,Alabama,22.0,648.0,486.0,626.0
24,Alaska,1.0,47762.0,53982.0,59179.0
26,Alaska,3.0,5925.0,7104.0,7491.0
31,Alaska,8.0,8707.0,9454.0,10261.0
37,Alaska,14.0,30348.0,34608.0,38299.0
45,Alaska,22.0,2782.0,2816.0,3129.0


In [4]:
# Unpivot the table to convert year columns (2020, 2021, 2022) into rows
melted_df = pd.melt(source_df, id_vars=["GeoName", "LineCode"], var_name="Year", value_name="Value")

melted_df.head(10)

,GeoName,LineCode,Year,Value
0,Alabama,1.0,2020,34878.0
1,Alabama,3.0,2020,4247.0
2,Alabama,8.0,2020,8883.0
3,Alabama,14.0,2020,21099.0
4,Alabama,22.0,2020,648.0
5,Alaska,1.0,2020,47762.0
6,Alaska,3.0,2020,5925.0
7,Alaska,8.0,2020,8707.0
8,Alaska,14.0,2020,30348.0
9,Alaska,22.0,2020,2782.0


In [5]:
# Pivot the table to make each unique LineCode a column
bea_personal_consumption_df = melted_df.pivot_table(index=["GeoName", "Year"], columns="LineCode", values="Value", aggfunc='first').reset_index()

bea_personal_consumption_df.head(10)

LineCode,GeoName,Year,1.0,3.0,8.0,14.0,22.0
0,Alabama,2020,34878.0,4247.0,8883.0,21099.0,648.0
1,Alabama,2021,39174.0,5186.0,10088.0,23412.0,486.0
2,Alabama,2022,42391.0,5500.0,10968.0,25297.0,626.0
3,Alaska,2020,47762.0,5925.0,8707.0,30348.0,2782.0
4,Alaska,2021,53982.0,7104.0,9454.0,34608.0,2816.0
5,Alaska,2022,59179.0,7491.0,10261.0,38299.0,3129.0
6,Arizona,2020,40605.0,5268.0,8974.0,25616.0,748.0
7,Arizona,2021,46023.0,6511.0,10059.0,28806.0,647.0
8,Arizona,2022,50123.0,6816.0,11156.0,31343.0,808.0
9,Arkansas,2020,34979.0,4185.0,8412.0,21048.0,1334.0


In [6]:
bea_personal_consumption_df.columns

Index(['GeoName', 'Year', 1.0, 3.0, 8.0, 14.0, 22.0], dtype='object', name='LineCode')

In [7]:
# Rename columns
bea_personal_consumption_df.rename(columns={"GeoName": "state_name",
                                            "Year": "year",
                                            1.0: "bea_pce_total",
                                            3.0: "bea_pce_durable_goods",
                                            8.0: "bea_pce_nondurable_goods",
                                            14.0: "bea_pce_services",
                                            22.0: "bea_pce_npish"}, inplace=True)

bea_personal_consumption_df.head(10)

LineCode,state_name,year,bea_pce_total,bea_pce_durable_goods,bea_pce_nondurable_goods,bea_pce_services,bea_pce_npish
0,Alabama,2020,34878.0,4247.0,8883.0,21099.0,648.0
1,Alabama,2021,39174.0,5186.0,10088.0,23412.0,486.0
2,Alabama,2022,42391.0,5500.0,10968.0,25297.0,626.0
3,Alaska,2020,47762.0,5925.0,8707.0,30348.0,2782.0
4,Alaska,2021,53982.0,7104.0,9454.0,34608.0,2816.0
5,Alaska,2022,59179.0,7491.0,10261.0,38299.0,3129.0
6,Arizona,2020,40605.0,5268.0,8974.0,25616.0,748.0
7,Arizona,2021,46023.0,6511.0,10059.0,28806.0,647.0
8,Arizona,2022,50123.0,6816.0,11156.0,31343.0,808.0
9,Arkansas,2020,34979.0,4185.0,8412.0,21048.0,1334.0


In [8]:
# Save to file
bea_personal_consumption_df.to_csv(Path("./source/bea_personal_consumption.csv"), index=False, header=True)

# BEA Income and Population

In [9]:
bea_income_population_csv = Path("./source/BEA/Income_population_by_county_2020_2022.csv")

# Skip the first 3 lines
source_df = pd.read_csv(bea_income_population_csv, header=3)

# Remove rows with NaN, since they are related to desciptions
source_df.dropna(inplace=True)

source_df.head()

,GeoFips,GeoName,LineCode,Description,2020,2021,2022
1,01001,"Autauga, AL",10.0,Personal income (thousands of dollars),2659457,2896204,2951555
2,01001,"Autauga, AL",20.0,Population (persons) 3,58902,59210,59759
4,01003,"Baldwin, AL",10.0,Personal income (thousands of dollars),11947762,13371784,13984484
5,01003,"Baldwin, AL",20.0,Population (persons) 3,233219,239361,246435
7,01005,"Barbour, AL",10.0,Personal income (thousands of dollars),926302,1001067,1002082


In [10]:
# Remove unnecessary columns
source_df.drop(columns=["GeoName", "Description"], inplace=True)

# Update data type
source_df["GeoFips"] = source_df["GeoFips"].astype(int)

source_df.head()

,GeoFips,LineCode,2020,2021,2022
1,1001,10.0,2659457,2896204,2951555
2,1001,20.0,58902,59210,59759
4,1003,10.0,11947762,13371784,13984484
5,1003,20.0,233219,239361,246435
7,1005,10.0,926302,1001067,1002082


In [11]:
# Unpivot the table to convert year columns (2020, 2021, 2022) into rows
melted_df = pd.melt(source_df, id_vars=["GeoFips", "LineCode"], var_name="Year", value_name="Value")

melted_df.head()

,GeoFips,LineCode,Year,Value
0,1001,10.0,2020,2659457
1,1001,20.0,2020,58902
2,1003,10.0,2020,11947762
3,1003,20.0,2020,233219
4,1005,10.0,2020,926302


In [12]:
# Pivot the table to make each unique LineCode a column
bea_income_population_df = melted_df.pivot_table(index=["GeoFips", "Year"], columns="LineCode", values="Value", aggfunc='first').reset_index()

bea_income_population_df.head(10)

LineCode,GeoFips,Year,10.0,20.0
0,1001,2020,2659457,58902
1,1001,2021,2896204,59210
2,1001,2022,2951555,59759
3,1003,2020,11947762,233219
4,1003,2021,13371784,239361
5,1003,2022,13984484,246435
6,1005,2020,926302,24960
7,1005,2021,1001067,24539
8,1005,2022,1002082,24706
9,1007,2020,775032,22183


In [13]:
# Rename columns
bea_income_population_df.rename(columns={"GeoFips": "county_fips",
                                         "Year": "year",
                                         10.0: "bea_total_income",
                                         20.0: "population"}, inplace=True)

# Filter out all records with population = (NA), use ~ as the NOT() condition
bea_income_population_df = bea_income_population_df[~(bea_income_population_df["population"] == "(NA)")]

# Calculate total income by multiplying by 1000
bea_income_population_df["bea_total_income"] = bea_income_population_df["bea_total_income"].astype("Int64") * 1000

bea_income_population_df.head()

LineCode,county_fips,year,bea_total_income,population
0,1001,2020,2659457000,58902
1,1001,2021,2896204000,59210
2,1001,2022,2951555000,59759
3,1003,2020,11947762000,233219
4,1003,2021,13371784000,239361


In [14]:
# Save to file
bea_income_population_df.to_csv(Path("./source/bea_income_population.csv"), index=False, header=True)